In [9]:
import base64
import requests
import time

In [12]:
def check_api_available():
    try:
        res = requests.get(f"{BASE_URL}/docs")
        return res.status_code == 200
    except requests.ConnectionError as e:
        print("Connection failed:", e)
        return False

def generate_image(prompt="a futuristic city at sunset"):
    # Step 1: Define a simple text-to-image graph
    payload = {
        "prompt": {
            "id": "txt2img_graph",
            "nodes": {
                "text2img": {
                    "type": "text_to_image",
                    "prompt": prompt,
                    "width": 512,
                    "height": 512,
                    "steps": 30,
                    "cfg_scale": 7.5,
                    "scheduler": "ddim"
                }
            }
        }
    }

    # Step 2: Submit graph
    res = requests.post(f"{BASE_URL}/prompt", json=payload)
    res.raise_for_status()
    graph_id = res.json()["graph_execution_state_id"]
    print(f"Submitted graph: {graph_id}")

    # Step 3: Poll for result
    while True:
        time.sleep(1)
        state = requests.get(f"{BASE_URL}/queue/status/{graph_id}")
        state.raise_for_status()
        data = state.json()
        if data["status"] == "succeeded":
            print("Image generation succeeded.")
            break
        elif data["status"] == "failed":
            raise RuntimeError("Generation failed.")
        else:
            print("Still running...")

    # Step 4: Get the image URL from the history
    history = requests.get(f"{BASE_URL}/history/{graph_id}")
    history.raise_for_status()
    images = history.json()["outputs"]
    image_url = images[0]["image"]["url"]

    image_data = requests.get(f"http://{DESKTOP_IP}:{PORT}{image_url}")
    with open("out.png", "wb") as f:
        f.write(image_data.content)

    print("Saved as out.png")

In [7]:
DESKTOP_IP = "192.168.0.171"
PORT = 9091
BASE_URL = f"http://{DESKTOP_IP}:{PORT}"

print("API reachable:", check_api_available())

API reachable: True


In [13]:
generate_image()

HTTPError: 405 Client Error: Method Not Allowed for url: http://192.168.0.171:9091/prompt

In [14]:
BASE = "http://192.168.0.171:9091/api/v1"
ENQUEUE_URL = f"{BASE}/queue/default/enqueue_batch"
QUEUE_STATUS_URL = f"{BASE}/queue/default/current"
HISTORY_URL = f"{BASE}/history"

def generate_image(prompt: str = "a futuristic city at sunset"):
    # --- GRAPH DEFINITION ---
    batch_payload = {
        "batch": {
            "batch_id": "batch_001",
            "origin": "api",
            "destination": "gallery",
            "graph": {
                "id": "graph_001",
                "nodes": {
                    "text_prompt": {
                        "id": "text_prompt",
                        "type": "compel",
                        "prompt": prompt
                    },
                    "neg_prompt": {
                        "id": "neg_prompt",
                        "type": "compel",
                        "prompt": ""
                    },
                    "noise": {
                        "id": "noise",
                        "type": "noise",
                        "seed": 42,
                        "width": 512,
                        "height": 512,
                        "use_cpu": False
                    },
                    "model_loader": {
                        "id": "model_loader",
                        "type": "main_model_loader",
                        "model": {
                            "base": "sd-1",  # or "sdxl" / "sd-3"
                            "type": "main",  # always "main"
                            "name": "v1-5-pruned-emaonly"  # must match installed model name
                        }
                    },
                    "denoise": {
                        "id": "denoise",
                        "type": "denoise_latents",
                        "steps": 25,
                        "cfg_scale": 7.5,
                        "scheduler": "ddim"
                    },
                    "output": {
                        "id": "output",
                        "type": "l2i",
                        "width": 512,
                        "height": 512,
                        "fp32": True
                    }
                },
                "edges": [
                    {"source": {"node_id": "model_loader", "field": "unet"}, "destination": {"node_id": "denoise", "field": "unet"}},
                    {"source": {"node_id": "text_prompt", "field": "conditioning"}, "destination": {"node_id": "denoise", "field": "positive_conditioning"}},
                    {"source": {"node_id": "neg_prompt", "field": "conditioning"}, "destination": {"node_id": "denoise", "field": "negative_conditioning"}},
                    {"source": {"node_id": "noise", "field": "noise"}, "destination": {"node_id": "denoise", "field": "noise"}},
                    {"source": {"node_id": "denoise", "field": "latents"}, "destination": {"node_id": "output", "field": "latents"}},
                    {"source": {"node_id": "model_loader", "field": "vae"}, "destination": {"node_id": "output", "field": "vae"}}
                ]
            },
            "runs": 1
        }
    }

    # --- ENQUEUE TASK ---
    res = requests.post(ENQUEUE_URL, json=batch_payload)
    res.raise_for_status()
    print("✅ Job enqueued.")

    # --- WAIT FOR RESULT ---
    print("⏳ Waiting for job to complete...")
    while True:
        r = requests.get(QUEUE_STATUS_URL)
        r.raise_for_status()
        data = r.json()
        if data["status"] == "succeeded":
            print("✅ Generation complete.")
            break
        elif data["status"] == "failed":
            raise RuntimeError("🚨 Image generation failed.")
        time.sleep(1)

    # --- GET FINAL IMAGE URL ---
    batch_id = batch_payload["batch"]["batch_id"]
    hist = requests.get(f"{HISTORY_URL}/{batch_id}")
    hist.raise_for_status()
    images = hist.json().get("outputs", [])
    if not images:
        raise ValueError("No image found in history.")
    image_url = images[0]["image"]["url"]

    # --- DOWNLOAD IMAGE ---
    img = requests.get(f"http://192.168.0.171:9091{image_url}")
    with open("out.png", "wb") as f:
        f.write(img.content)
    print("📸 Image saved as out.png")

generate_image()

HTTPError: 422 Client Error: Unprocessable Entity for url: http://192.168.0.171:9091/api/v1/queue/default/enqueue_batch